<a href="https://colab.research.google.com/github/ClouDevops101/size-limit/blob/main/recommendation-engine-udemy-v4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# build a online courses recommendation system

In [ ]:
# importing libraries
# src https://www.youtube.com/watch?v=ueKXSupHz6Q

In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [131]:
# show available language
print(stopwords.fileids())

['arabic', 'azerbaijani', 'danish', 'dutch', 'english', 'finnish', 'french', 'german', 'greek', 'hungarian', 'indonesian', 'italian', 'kazakh', 'nepali', 'norwegian', 'portuguese', 'romanian', 'russian', 'slovene', 'spanish', 'swedish', 'tajik', 'turkish']


In [ ]:
# Load the data 
from google.colab import files
uploaded = files.upload()

KeyboardInterrupt: ignored

In [2]:
#Store the data 
df = pd.read_csv('udemy-data-v2.csv')
# show the first 3 rows of data 
df.head(3)

,Course_id,title,description,headline,category,subcategory,instructors_name,instructors_job_title,price,num_subscribers,avg_rating,num_reviews,created
0,1117836,NoSQL Databases For Beginners,<p>NoSQL (Non-SQL or Not-only-SQL) databases a...,Learn to Store and Retrieve Data from a Datab...,Development,Database Design & Development,Bluelime Learning Solutions,Learning made simple,€94.99,1178,3.5,128,2017-02-17T12:46:11Z
1,1117864,Tai Chi Foundation Course,<p>Whether you're a beginner or experienced st...,Traditional methods. Modern teaching.,Health & Fitness,General Health,Aamir Rafi,Tai Chi practitioner & life long martial artist,€89.99,5,0.0,0,2017-02-17T13:28:15Z
2,1117988,Music Pedagogy: 7 advices to teach a musical i...,<p><strong>Are you a music teacher?</strong></...,Learn how to organize and improve your classes.,Music,Music Fundamentals,Ilse Lozoya,"University Professor, Piano Teacher, Course De...",€39.99,1345,4.0,7,2017-02-17T15:50:16Z


In [3]:
df.shape


(645, 13)

In [4]:
# Creat a list of important columns for the recommendation engine
columns= ['title','description','headline','category','subcategory','instructors_name','instructors_job_title']

In [5]:
# show the data 
df[columns].head(3)


,title,description,headline,category,subcategory,instructors_name,instructors_job_title
0,NoSQL Databases For Beginners,<p>NoSQL (Non-SQL or Not-only-SQL) databases a...,Learn to Store and Retrieve Data from a Datab...,Development,Database Design & Development,Bluelime Learning Solutions,Learning made simple
1,Tai Chi Foundation Course,<p>Whether you're a beginner or experienced st...,Traditional methods. Modern teaching.,Health & Fitness,General Health,Aamir Rafi,Tai Chi practitioner & life long martial artist
2,Music Pedagogy: 7 advices to teach a musical i...,<p><strong>Are you a music teacher?</strong></...,Learn how to organize and improve your classes.,Music,Music Fundamentals,Ilse Lozoya,"University Professor, Piano Teacher, Course De..."


In [6]:
# check for missing values in the important columns
df[columns].isnull().values.any()

# data cleaning is needed

True

In [7]:
# Create a function to combine the values of the important columns into a single string
def get_important_features(data):
  important_features = []
  for i in range(0, data.shape[0]):
    important_features.append(str(data['title'][i])+' '+str(data['description'][i])+' '+str(data['headline'][i])+' '+str(data['category'][i])+' '+str(data['subcategory'][i])+' '+str(data['instructors_name'][i])+' '+str(data['instructors_job_title'][i]) )
  return important_features


In [8]:
# Create a column to hold the combined strings
df['important_features'] = get_important_features(df)

#show the data 
df.head(10)

,Course_id,title,description,headline,category,subcategory,instructors_name,instructors_job_title,price,num_subscribers,avg_rating,num_reviews,created,important_features
0,1117836,NoSQL Databases For Beginners,<p>NoSQL (Non-SQL or Not-only-SQL) databases a...,Learn to Store and Retrieve Data from a Datab...,Development,Database Design & Development,Bluelime Learning Solutions,Learning made simple,€94.99,1178,3.500000,128,2017-02-17T12:46:11Z,NoSQL Databases For Beginners <p>NoSQL (Non-SQ...
1,1117864,Tai Chi Foundation Course,<p>Whether you're a beginner or experienced st...,Traditional methods. Modern teaching.,Health & Fitness,General Health,Aamir Rafi,Tai Chi practitioner & life long martial artist,€89.99,5,0.000000,0,2017-02-17T13:28:15Z,Tai Chi Foundation Course <p>Whether you're a ...
2,1117988,Music Pedagogy: 7 advices to teach a musical i...,<p><strong>Are you a music teacher?</strong></...,Learn how to organize and improve your classes.,Music,Music Fundamentals,Ilse Lozoya,"University Professor, Piano Teacher, Course De...",€39.99,1345,4.000000,7,2017-02-17T15:50:16Z,Music Pedagogy: 7 advices to teach a musical i...
3,1118012,Ultimate Power Electronics for Electrical Powe...,"<p><strong>""Ultimate Power Electronics Course ...",Learn everything about power electronics for e...,Teaching & Academics,Engineering,Ahmed Mahdy / Khadija Academy,Electrical Power Engineer,€94.99,3186,4.592593,481,2017-02-17T16:15:02Z,Ultimate Power Electronics for Electrical Powe...
4,1118046,Facebook Advertising: Create Converting Facebo...,<p><strong>Over 8000 students have joined this...,"Learn how to select your niche, identify marke...",Marketing,Social Media Marketing,Barry North,"Award Winning ""Sexiest Voice""",€89.99,10498,4.200000,68,2017-02-17T16:48:28Z,Facebook Advertising: Create Converting Facebo...
5,1118072,Planning & Scheduling: Be the Professional fro...,<p><strong>Planning and Scheduling</strong> is...,Project Planning and Scheduling Management cou...,Business,Project Management,"Construction Project Management Guru (PSP, PMI...",Promoting Professionalism in Planning and Sche...,€109.99,3393,4.074074,615,2017-02-17T17:08:03Z,Planning & Scheduling: Be the Professional fro...
6,1118146,Wie du einen genialen Vortrag hältst,<p>Dein Vortrag ist deine Chance</p> <p>Mit d...,Mit Simon (Profi-Speaker) und Michi (NLP-Train...,Personal Development,Career Development,Simon Schnetzer & Michael Schmidt,Speaker & Persönlichkeitsentwickler,€44.99,345,4.100000,31,2017-02-17T17:50:36Z,Wie du einen genialen Vortrag hältst <p>Dein V...
7,1118268,Top Five Leadership Behaviors,<p>Every Leader Needs To Gain Their Team's Tru...,Strong Leaders Don't Just Happen: Learn These ...,Personal Development,Leadership,Donna Galatas,HR Vice Empress / CEO of The Galatas Group,€94.99,129,4.350000,27,2017-02-17T19:38:01Z,Top Five Leadership Behaviors <p>Every Leader ...
8,1118274,The Visual Guide on How Neural Networks Learn ...,<p>Course Achievements (January 2021):&nbsp;</...,The BEST Resource for Understanding Neural Net...,Development,Data Science,Mauricio Maroto,"+10,000 students and growing!",€94.99,3163,4.700000,143,2017-02-17T19:54:28Z,The Visual Guide on How Neural Networks Learn ...
9,1118300,Aprende a crear Plugins para WordPress desde c...,<p><strong>Más de 60 reseñas fantásticas !!! 4...,Es hora de saltar al siguiente paso en WordPre...,Development,Web Development,Gilbert Rodríguez,Desarrollador Web,€94.99,1457,4.500000,243,2017-02-17T20:32:46Z,Aprende a crear Plugins para WordPress desde c...


In [9]:
# creating stop word
stop_words =  stopwords.words('arabic') + stopwords.words('azerbaijani') + stopwords.words('danish') + stopwords.words('dutch') + stopwords.words('english') + stopwords.words('finnish') + stopwords.words('french') + stopwords.words('german') + stopwords.words('greek') + stopwords.words('hungarian') + stopwords.words('indonesian') + stopwords.words('italian') + stopwords.words('kazakh') + stopwords.words('nepali') + stopwords.words('norwegian') + stopwords.words('portuguese') + stopwords.words('romanian') + stopwords.words('russian') + stopwords.words('slovene') + stopwords.words('spanish') + stopwords.words('swedish') + stopwords.words('tajik') + stopwords.words('turkish')
print(stop_words)

['إذ', 'إذا', 'إذما', 'إذن', 'أف', 'أقل', 'أكثر', 'ألا', 'إلا', 'التي', 'الذي', 'الذين', 'اللاتي', 'اللائي', 'اللتان', 'اللتيا', 'اللتين', 'اللذان', 'اللذين', 'اللواتي', 'إلى', 'إليك', 'إليكم', 'إليكما', 'إليكن', 'أم', 'أما', 'أما', 'إما', 'أن', 'إن', 'إنا', 'أنا', 'أنت', 'أنتم', 'أنتما', 'أنتن', 'إنما', 'إنه', 'أنى', 'أنى', 'آه', 'آها', 'أو', 'أولاء', 'أولئك', 'أوه', 'آي', 'أي', 'أيها', 'إي', 'أين', 'أين', 'أينما', 'إيه', 'بخ', 'بس', 'بعد', 'بعض', 'بك', 'بكم', 'بكم', 'بكما', 'بكن', 'بل', 'بلى', 'بما', 'بماذا', 'بمن', 'بنا', 'به', 'بها', 'بهم', 'بهما', 'بهن', 'بي', 'بين', 'بيد', 'تلك', 'تلكم', 'تلكما', 'ته', 'تي', 'تين', 'تينك', 'ثم', 'ثمة', 'حاشا', 'حبذا', 'حتى', 'حيث', 'حيثما', 'حين', 'خلا', 'دون', 'ذا', 'ذات', 'ذاك', 'ذان', 'ذانك', 'ذلك', 'ذلكم', 'ذلكما', 'ذلكن', 'ذه', 'ذو', 'ذوا', 'ذواتا', 'ذواتي', 'ذي', 'ذين', 'ذينك', 'ريث', 'سوف', 'سوى', 'شتان', 'عدا', 'عسى', 'عل', 'على', 'عليك', 'عليه', 'عما', 'عن', 'عند', 'غير', 'فإذا', 'فإن', 'فلا', 'فمن', 'في', 'فيم', 'فيما', 'فيه', 'فيها', '

In [10]:
# convert the text to a matrix of token counts 
df['important_features'].replace('<[^<]+?>',' ',regex=True, inplace = True)
print (df['important_features'].head(3))
df['important_features'] = df['important_features'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))
cm = CountVectorizer().fit_transform(df['important_features'])

# GEt the cosine similarity matrix from the count matrix
cs = cosine_similarity(cm)
#Print the cosine similarity matrix
print(cs)


0    NoSQL Databases For Beginners  NoSQL (Non-SQL ...
1    Tai Chi Foundation Course  Whether you're a be...
2    Music Pedagogy: 7 advices to teach a musical i...
Name: important_features, dtype: object
[[1.         0.08476291 0.14103213 ... 0.11829838 0.         0.23462534]
 [0.08476291 1.         0.17646815 ... 0.14558425 0.         0.06099894]
 [0.14103213 0.17646815 1.         ... 0.19714449 0.         0.12309373]
 ...
 [0.11829838 0.14558425 0.19714449 ... 1.         0.         0.1348569 ]
 [0.         0.         0.         ... 0.         1.         0.        ]
 [0.23462534 0.06099894 0.12309373 ... 0.1348569  0.         1.        ]]


In [11]:
# Get the shape of the cosine similarity matrix 
cs.shape


(645, 645)

In [20]:
# Get the title of the course that the user likes 
title_to_search = 'Beginner Data Analysis Using SQL and Microsoft SQL Server'

# Find the course id
course_id = df[df.title == title_to_search]['Course_id'].index
print (course_id[0])
#course_id = 251

450


In [21]:
# Create a list of enumeration for the similarity score [(course_id, similiraty score ),(..)]
scores = list(enumerate(cs[course_id[0]]))

In [22]:
#sort the list 
sorted_scores = sorted(scores, key = lambda x:x[1], reverse  = True)
sorted_scores = sorted_scores[1:]


In [23]:
#Print the sorted scores
print(sorted_scores)

[(96, 0.7684947407547379), (93, 0.6676396803224743), (451, 0.5002982846894393), (0, 0.4761863009446829), (238, 0.462199195258419), (488, 0.46178997833556035), (95, 0.44578511914786173), (311, 0.40728408062957294), (203, 0.40293825029933), (348, 0.3646211257691806), (146, 0.3337613352091694), (261, 0.32816166234553934), (644, 0.3097427036232282), (170, 0.282467377273899), (343, 0.26471837423372957), (84, 0.2572590554771448), (531, 0.24786306271977274), (22, 0.24550651952834218), (457, 0.24260852343677922), (234, 0.2379272409556267), (229, 0.23331457818745363), (563, 0.2216454803196132), (73, 0.21994076777852398), (362, 0.21645247030087902), (252, 0.21601554504621778), (44, 0.2143383258282074), (373, 0.21361878157351166), (470, 0.21157305544372565), (588, 0.20685498813171996), (603, 0.20353274876272984), (418, 0.19708693703340172), (542, 0.1968544657913625), (143, 0.19181142828268138), (279, 0.183777294604954), (262, 0.18361669878930714), (281, 0.18293640217073784), (367, 0.1813660012342

In [24]:
# Create a loop to print the first 7 similar movies
j = 0
print('The 7 most recommended courses : ', title_to_search, '\nare:\n')
print('ID  Course Title' )
for item in sorted_scores:
  course_title = df[df.index == item[0]]['title'].values[0]
  course_id = df[df.index == item[0]]['Course_id'].values[0]
  #course_headline = df[df.index == item[0]]['headline'].values[0]
  num_subscribers = df[df.index == item[0]]['num_subscribers'].values[0]
  print(course_id, course_title, " | " + str(num_subscribers) )
  j=j+1
  if j>20:
    break

The 7 most recommended courses :  Beginner Data Analysis Using SQL and Microsoft SQL Server 
are:

ID  Course Title
1121780 Learn to use SQL for Data Analysis and Reporting  | 1632
1121702 Learn Microsoft SSIS (SQL Server Integration Services)  | 826
1135322 Introduction To SQL Server Reporting Services -SSRS  | 603
1117836 NoSQL Databases For Beginners  | 1178
1126478 Student Registration Form in C# and SQL Server in Urdu  | 14
1136680 SQL Server 101 : Microsoft SQL Server for Absolute Beginners  | 576
1121716 Introduction to SQL Server Reporting Services (SSRS)  | 598
1129804 Haskell: Data Analysis Made Easy  | 604
1125010 Oracle SQL- Ultimate In-Depth : Beginner to Advance  | 1877
1131092 R: Complete Machine Learning Solutions  | 564
1123198 Colors for Data Science A-Z: Data Visualization Color Theory  | 5754
1127462 Microsoft Power Pivot (Excel) and SSAS (Tabular DAX model)  | 2871
1142202 Master Angular 4 by Example - Build 7 Awesome Apps!  | 1484
1124030 Bash Shell Programming fo